# How to programmatically generate and execute an IPython notebook

This is a response to [this question on StackOverflow](http://stackoverflow.com/questions/22328052/ipython-notebook-programatically-read-and-execute-cells). It illustrates how to generate an IPython notebook programmatically by generating markdown and code cells on the fly, and how to execute the resulting notebook.

The automatically generated notebook which this code produces can be seen [here]().

This code has been tested with IPython version 3.1. It may not work with different versions, since the `nbconvert` machinery is in constant flux, and the `nbconvert` is currently being moved to a [separate Jupyter repository](https://github.com/jupyter/jupyter_nbconvert) anyway.

In [1]:
import IPython
print("Using IPython version {}".format(IPython.__version__))

Using IPython version 3.1.0


## Generating a new notebook with some code cells

### Creating an empty notebook

First we create a new IPython notebook using the current nbformat version 4.

In [2]:
from IPython import nbformat

In [3]:
nb = nbformat.v4.new_notebook()

The notebook contains an attribute `nb.cells` which initially is an emtpy list.

In [4]:
nb.cells

[]

Alternatively, we could have loaded the notebook from an existing `.ipynb` file via:

    nb = nbformat.read(filename, as_version=4)

In this case `nb.cells` would be a list containing the existing cells. We could then append/insert new cells into this list, or even overwrite existing cells with new content by setting `nb.cells[i] = ...`.

### Adding a markdown cell

We begin filling the list of cells by generating a markdown cell and appending it to `nb.cells`.

In [5]:
markdown_cell = nbformat.v4.new_markdown_cell("This notebook was automatically generated.")

nb.cells.append(markdown_cell)

In [6]:
nb.cells

[{'cell_type': 'markdown',
  'metadata': {},
  'source': 'This notebook was automatically generated.'}]

### Adding a couple of code cells

Next we generate a code cell which simply prints "Hello world", and also append it to the list of cells.

In [7]:
source_1 = "print('Hello world')"
code_cell_1 = nbformat.v4.new_code_cell(source=source_1)
nb.cells.append(code_cell_1)

Note that when printing the list of cells, the `outputs` field of the code cell is currently empty because the cell has not been executed yet (this will happen below).

In [8]:
nb.cells

[{'cell_type': 'markdown',
  'metadata': {},
  'source': 'This notebook was automatically generated.'},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [],
  'source': "print('Hello world')"}]

Generating multi-line code cells and injecting values on-the-fly works pretty much the same way. The following creates two code cells which define a variable `frequency` and display a sine wave with this frequency using matplotlib.

Note that we inject the frequency value into the source of the code cell on the fly. In a real-world example this would allow us to produce a parameter study with one automatically generated notebook documenting the outcome for each parameter value.

In [9]:
import textwrap

freq = 5.0  # frequency of the sine wave to be displayed by the code cells below

source_2 = "frequency = {:.1f}".format(freq)

source_3 = textwrap.dedent("""\
    %matplotlib inline
    import matplotlib.pyplot as plt
    import numpy as np

    x = np.linspace(0, 2*np.pi, 200)
    y = np.sin(frequency*x)

    plt.plot(x, y)
    """)

code_cell_2 = nbformat.v4.new_code_cell(source=source_2)
code_cell_3 = nbformat.v4.new_code_cell(source=source_3)

nb.cells.append(code_cell_2)
nb.cells.append(code_cell_3)

In [10]:
nb.cells

[{'cell_type': 'markdown',
  'metadata': {},
  'source': 'This notebook was automatically generated.'},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [],
  'source': "print('Hello world')"},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [],
  'source': 'frequency = 5.0'},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [],
  'source': '%matplotlib inline\nimport matplotlib.pyplot as plt\nimport numpy as np\n\nx = np.linspace(0, 2*np.pi, 200)\ny = np.sin(frequency*x)\n\nplt.plot(x, y)\n'}]

We can take a look at the JSON code generated so far:

In [11]:
print(nbformat.v4.writes_json(nb))

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This notebook was automatically generated."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print('Hello world')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "frequency = 5.0"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%matplotlib inline\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "\n",
    "x = np.linspace(0, 2*np.pi, 200)\n",
    "y = np.sin(frequency*x)\n",
    "\n",
    "plt.plot(x, y)\n"
   ]
  }
 ],
 "metadata": {},
 "nbformat": 4,
 "nbformat_minor": 0
}


If we want to share this notebook (or inspect it for debugging), we can write it out to a `.ipynb` file.

In [12]:
with open('automatically_generated_notebook.ipynb', 'w') as f:
    f.write(nbformat.v4.writes_json(nb))

In [13]:
!ls automatically_generated_notebook.ipynb

automatically_generated_notebook.ipynb


# Executing the newly generated notebook

In order to execute the notebook `nb` which we generated above we first need to create a new instance of `ExecutePreprocessor`. Here we set a timeout for execution of code cells of 30 seconds. If you have very long-running cells you may want to increase this.

In [14]:
from IPython.nbconvert.preprocessors.execute import ExecutePreprocessor

In [15]:
pp = ExecutePreprocessor()
pp.timeout = 30  # seconds
pp.interrupt_on_timeout = True

Then we can execute the notebook by calling `pp.preprocess()`. I'm not 100% sure what the role of the `resources` dictionary is, but it needs to be supplied or else `preprocess()` will produce an error.

In [16]:
nb_executed, resources = pp.preprocess(nb, resources={})

Let's write the executed notebook to a file. Note that we include the value of `frequency` in the filename. This may be useful if we'd like to run the same "template" notebook for multiple values of `frequency`, e.g. for a parameter study with automatically generated reports.

In [17]:
filename = 'executed_notebook_with_freq_{:.1f}.ipynb'.format(freq)
print("Filename of executed notebook: '{}'".format(filename))

Filename of executed notebook: 'executed_notebook_with_freq_5.0.ipynb'


In [18]:
with open(filename, 'w') as f:
    f.write(nbformat.v4.writes_json(nb_executed))

Taking a look at the first few lines of the saved `.ipynb` file, we notice that the `outputs` field of the code cells have now been filled with the computed results.

In [19]:
!head -n 20 executed_notebook_with_freq_5.0.ipynb

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This notebook was automatically generated."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Hello world\n"
     ]


To view the contents of the automatically generated notebook, see generated notebook, see [here]().